In [25]:
import pandas as pd
import numpy as np
import requests


from bs4 import BeautifulSoup

In [82]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh;Intel MacOS X 10_11_5) Apple WebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537,36'}

page = requests.get(url, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')


In [7]:
products = soup.find('ul', class_='products-listing small')

In [8]:
#finding products in HTML
product_list = products.find_all('article', class_='hm-product-item')

# product_id
product_id = [p.get('data-articlecode') for p in product_list]

#product_category
product_category = [p.get('data-category') for p in product_list]

#product_name
product_list = products.find_all('a', class_= 'link')
product_name = [p.get_text() for p in product_list]

#product_price
product_list = products.find_all('span',class_ = 'price regular')
product_price = [p.get_text() for p in product_list]

In [20]:
#create the DataFrame
data = pd.DataFrame([product_id,product_category,product_name,product_price]).T
data.columns = ['product_id','product_category','product_name','product_price']

In [43]:
#identify how many product are
total_item = soup.find_all('h2',class_ = 'load-more-heading')[0].get('data-total')
#page_number = np.round(int(total_item)/36)
page_number

3.0

In [44]:
url02 = url + '?page-size=' + str(int(page_number*36))

In [119]:
url = 'https://www2.hm.com/en_us/productpage.0636207011.html'
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh;Intel MacOS X 10_11_5) Apple WebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537,36'}

page = requests.get(url, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')


In [114]:
#color_name
product_list = soup.find_all('a',class_='filter-option miniature')

color_name = [p.get('data-color') for p in product_list]
product_id = [p.get('data-articlecode') for p in product_list]

df_color = pd.DataFrame(product_id,color_name).reset_index()
df_color.columns = ['color_name','product_id']

#generate style id + color id
df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

In [115]:
#composition
product_composition_list = soup.find_all('div', class_ = 'pdp-description-list-item')
product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]

# rename dataframe
df_composition = pd.DataFrame(product_composition).T
df_composition.columns = df_composition.iloc[0]
# delete first row
df_composition = df_composition.iloc[1:].fillna(method='ffill')

#generate style id + color id
df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

In [120]:
# Table with information about color and composition
data_sku = pd.merge(df_color,df_composition[['style_id','Fit','Composition']],how='left', on='style_id')